In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import pandas as pd
from plots_file import *
from our_functionsv3 import *

np.random.seed(0)
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [6]:
X_train, Y_train = get_data(pos = "twitter-datasets/train_pos_preprocessed.txt", neg = "twitter-datasets/train_neg_preprocessed.txt")

In [7]:
Y_train_oh = convert_to_one_hot(Y_train, C=2)

In [134]:
X_train_tab = np.asarray(X_train)

In [8]:
size = []
for elem in X_train:
    size.append(len(elem.split()))
max_length = max(size)

In [9]:
max_length

45

In [10]:
ids, _ = get_test_data("twitter-datasets/test_data.txt")
X_test = read_data("twitter-datasets/test_preprocessed.txt")

In [11]:
X_test[0:20]

['sea doo pro sea scooter sports portable seascootersave air stay longer water url',
 'user shucks well work week ca come cheer oh put batteries calculator',
 'cant stay away bug thats baby',
 'user lol im perfectly fine contagious anymore lmao',
 'whenever fall asleep watching tv always wake headache',
 'user needs get rid thing scares lol need car either needs drivers ed',
 'whatever terrible mood',
 'yesss rt user user thanks jordan love gon na call later',
 'friend user text check last night',
 'user followback please ur unitytour come europe sweden',
 'watch dumb asses get lock today',
 'obsessed phasell user killed best album ever love yew roycee rt',
 'user robert de niro gay name like lewy understand hahahahha sherlock',
 'user canada grade since grade sucks',
 'user please say hi denmark would amazing user live url',
 'finally home',
 'custom picture frame poster frame wide complete gold frame frame manufactured url',
 'new followers mention followback boo',
 'user yep looks l

In [12]:
X_test = np.asarray(X_test)

In [13]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

#word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('./twitter/glove.twitter.27B.50d.txt')
#word_to_vec_map['0.45973'] a que 49 mots wtf. si quelqu'un comprend pourquoi je veux bien que on m'explique
#del word_to_vec_map['0.45973']
#del word_to_index['0.45973']

for word, index in word_to_index.items():
    if(len(word_to_vec_map[word]) != 50):
        print(index)
        print(word)
        print(len(word_to_vec_map[word]))

### Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences

In [14]:
def sentences_to_indices(X, word_to_index, max_length):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """    
    m = len(X)                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_length))
    
    for i in range(m):                               # loop over training examples        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [x.lower() for x in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            if w in word_to_index.keys():
            # Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
                j = j + 1
            else:
                pass
                #X_indices[i, j] = -1   #si il ne connait pas le mot il met -1, voir comment on gère ça après
                # Increment j to j + 1
                #print("{}   not in twitter dataset".format(w))
            #j = j + 1
        '''    
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
        '''
    return X_indices

In [15]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)

In [16]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_length)

In [17]:
X_test_indices[0:20]

array([[322569., 127727., 292667., 322569., 321903., 340038., 289109.,
         49059., 342181., 225401., 383772., 373871.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.],
       [374186., 330277., 385218., 389836., 384714.,  88789., 105625.,
         97845., 268508., 295763.,  70390.,  89640.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.],
       [ 91041., 342181.,  64331.,  86258., 357226.,  65526.,      0.,
     

In [236]:
X1_indices = sentences_to_indices(X_train[0:3],word_to_index, max_length = 15)
print("X1 =", X_train[0:3])
print("X1_indices =", X1_indices)

X1 = ['missing much time school omg never ill wan na get better', 'user thanks follow', 'chucky collection child play child play bride chucky dvd url']
X1_indices = [[246283. 251645. 359889. 321160. 269664. 259914. 187377. 382897. 255103.
  160418.  74597.      0.      0.      0.      0.]
 [374186. 357161. 151050.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.]
 [100501. 104990.  98970. 286375.  98970. 286375.  83954. 100501. 131420.
  373871.      0.      0.      0.      0.      0.]]


### Embedding() layer in Keras
- input:    output of sentences_to_indices()
- output:   word embeddings for a sentence

In [18]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    #print(vocab_len)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    #print(emb_matrix.shape)
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    #print(len(word_to_index  ))
    #print(len(word_to_vec_map))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False) 

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [19]:
#erreur peut etre due au fait que j'ai du enlever word_to_vec_map['0.45973'] (voir au dessus)
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


### LSTM layers
- input:  array of sentences of shape (m, max_len, ) 
- output: softmax probability vector of shape (m, C = 2)

In [20]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences = False, return_state = False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [21]:
model = Emojify_V2((max_length,), word_to_vec_map, word_to_index)
#model.summary()

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)
Y_train_oh = convert_to_one_hot(Y_train, C = 2)

In [ ]:
history_lstm = model.fit(X_train_indices, Y_train_oh, epochs = 30, batch_size = 32, validation_split = 0.1, shuffle=True)

Train on 180000 samples, validate on 20000 samples
Epoch 1/30
143136/180000 [======================>.......] - ETA: 2:38 - loss: 0.5208 - acc: 0.7241

In [ ]:
model.save_weights("model_lstm_20_11.h5")

In [ ]:
final_val_loss = history_lstm['val_loss'][-1]
final_val_acc  = history_lstm['val_acc'][-1]

In [ ]:
argmin = np.argmin(history_lstm['val_loss'])
early_val_loss = history_lstm['val_loss'][argmin]
early_val_acc  = history_lstm['val_acc'][argmin]

In [220]:
def smiley_LSTM(input_shape, word_to_vec_map, word_to_index, dropout_rate):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    
    # Add dropout with a probability of 0.5
    X = Dropout(dropout_rate)(X)
    
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences = False, return_state = False)(X)
    
    # Add dropout with a probability of 0.5
    X = Dropout(dropout_rate)(X)
    
    # Propagate X through a Dense layer with softmax activation to get back a batch of 2-dimensional vectors.
    X = Dense(2)(X)
    
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [221]:
Y_train_oh.shape
X_train_indices.shape

(200000, 45)

In [222]:
def complete_model(word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                   epochs = 50, loss ='categorical_crossentropy', optimizer ='adam'):
    
    model = smiley_LSTM((max_length,), word_to_vec_map, word_to_index, dropout_rate)
    
    if summary:
        model.summary()
        
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    print("Begin to fit")
    history = model.fit(X_train_indices, Y_train_oh, epochs = epochs, batch_size = 128, validation_split = 0.1, shuffle=True)
    
    return history, model

In [ ]:
history_lstm, model_lstm = complete_model(word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                                     epochs = 30, loss ='categorical_crossentropy', optimizer ='adam')

In [ ]:
model_lstm.save_weights("model_lstm.h5")

In [ ]:
plot_history(history_lstm, 'history of model 1')

In [ ]:
final_val_loss = history_lstm['val_loss'][-1]
final_val_acc  = history_lstm['val_acc'][-1]

In [ ]:
argmin = np.argmin(history_lstm['val_loss'])
early_val_loss = history_lstm['val_loss'][argmin]
early_val_acc  = history_lstm['val_acc'][argmin]

### Analysis of training

In [ ]:
# taken from my miniproject in ANN course
plot_history(history, 'history of model 1')

### Submission

In [328]:
y_pred = model.predict(X_test_indices)

In [330]:
label = np.zeros([X_test_indices.shape[0], 1])
for i in range(len(X_test_indices)):
    label[i] = np.argmax(y_pred[i])

In [331]:
len(label)

10000

In [332]:
label[0:20]

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.]])

In [333]:
label[label == 0] = -1

In [335]:
create_csv_submission(ids, label, 'submissions/submission_model_LSTM_basic.csv')